In [1]:
from pandas import read_csv
from collections import Counter

In [2]:
filepath = 'dataset.csv'
column_names = ["Tweet", "Label"]

In [3]:
# membuat list dengan isi tweet dan label dari dataset
df = read_csv(filepath, header=0, sep=',', names=column_names)
tweets = df.Tweet.to_list()
labels = df.Label.to_list()

print(df)

                                                 Tweet          Label
0    aaa kakk dah vaksin pulang mam minum obat dara...  Setuju Vaksin
1                             aaa tempat vaksin pulang  Setuju Vaksin
2    abis dpt vaksin sekolah ajar ptm ya nggak dari...  Setuju Vaksin
3                                          abis vaksin  Setuju Vaksin
4                             abis vaksin bawa ngantuk  Setuju Vaksin
..                                                 ...            ...
716                        yoon gue vaksin gue takuttt  Setuju Vaksin
717  yuk sobat sehat ajak anak vaksin covid lupa aj...  Setuju Vaksin
718                yuk vaksin jokowima ruf pimpin baik  Setuju Vaksin
719  zaman justifikasi benda justifikasi nilai juju...   Tidak Setuju
720  zoo kalo vaksin kucing domestik tuu berapa yaa...   Tidak Setuju

[721 rows x 2 columns]


In [4]:
words_ = ' '.join(tweets)
# buat list dengan isi semua kata yang ada di tweets
words = words_.split()
# hitung jumlah kata menggunakan class Counter
count_words = Counter(words)
words_count = len(count_words)
sorted_words = count_words.most_common(words_count)

print(words_count)
print(sorted_words)

2560
[('vaksin', 845), ('yg', 100), ('udah', 89), ('covid', 81), ('ga', 74), ('dosis', 73), ('ya', 59), ('anak', 59), ('gue', 48), ('vaksinasi', 47), ('abis', 43), ('aja', 43), ('kali', 42), ('kalo', 42), ('gak', 39), ('nya', 38), ('dah', 37), ('sinovac', 36), ('maju', 35), ('sertifikat', 33), ('pas', 31), ('aman', 30), ('sakit', 30), ('takut', 30), ('gua', 30), ('sehat', 29), ('indonesia', 29), ('udh', 28), ('perintah', 27), ('sih', 25), ('baik', 25), ('rakyat', 23), ('lo', 23), ('pimpin', 23), ('masuk', 22), ('habis', 22), ('izin', 22), ('jokowima', 22), ('ruf', 22), ('x', 22), ('pake', 21), ('efek', 21), ('orang', 21), ('masyarakat', 20), ('terima', 20), ('banget', 19), ('tau', 18), ('pcr', 18), ('bgt', 17), ('usia', 17), ('gw', 17), ('biar', 17), ('ayo', 17), ('cepat', 17), ('masker', 17), ('jam', 16), ('lupa', 16), ('suntik', 16), ('tangan', 15), ('tp', 15), ('pfizer', 15), ('warga', 15), ('klo', 15), ('senin', 15), ('wajib', 15), ('kasih', 15), ('pandemi', 14), ('jalan', 14), ('g